# BitFit 实战

## Step1 导入相关包

In [1]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 加载数据集

In [2]:
ds = Dataset.load_from_disk("./data/alpaca_data_zh/")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
ds[2266]

{'output': '英翼兽 (Ying Wing Beast)', 'input': '', 'instruction': '生成一个奇幻生物的名字'}

## Step3 数据集预处理

In [4]:
tokenizer = AutoTokenizer.from_pretrained("D:/pretrained_model/models--Langboat--bloom-1b4-zh")
tokenizer

BloomTokenizerFast(name_or_path='D:/pretrained_model/models--Langboat--bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
'''
    构架以下数据格式：
    Human：[instruction][input]

    Assistant：[response]

    input_ids、attention_mask和正文相同
    labels需要将assistant之前的设置为-100
'''
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func,  remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [7]:
print(tokenizer.decode(tokenized_ds["input_ids"][0]))

Human: 保持健康的三个提示。

Assistant: 以下是保持健康的三个提示：

1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。

2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。

3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>


In [8]:
print((tokenized_ds['labels'][0]))

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 5040, 584, 7160, 28215, 9905, 15211, 1038, 189, 189, 20, 17, 210, 7160, 9239, 3365, 420, 9487, 1965, 15342, 9239, 6053, 355, 1227, 32998, 554, 34487, 1326, 17632, 355, 975, 5189, 39177, 6455, 355, 12144, 17335, 8481, 355, 1437, 12785, 6698, 23294, 672, 189, 21, 17, 210, 22905, 15356, 420, 9487, 15597, 22985, 373, 19320, 554, 18524, 554, 1229, 6663, 1467, 642, 16422, 18235, 27624, 17520, 8732, 355, 8729, 1246, 7110, 554, 1246, 16422, 642, 15224, 11367, 355, 718, 7160, 28215, 15356, 12577, 672, 189, 22, 17, 210, 20027, 18399, 420, 20027, 1079, 15713, 6455, 21230, 355, 34810, 9487, 1688, 8637, 967, 3958, 210, 43738, 20027, 420, 15822, 20027, 12785, 17374, 12115, 355, 5189, 9239, 8113, 355, 1437, 5706, 4202, 19112, 16610, 1276, 420, 2]


In [9]:
print(tokenizer.decode(list(filter(lambda x: x!=-100, tokenized_ds['labels'][1]))))

4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>


## Step4 创建模型

In [10]:
model = AutoModelForCausalLM.from_pretrained("D:/pretrained_model/models--Langboat--bloom-1b4-zh", low_cpu_mem_usage=True)

In [11]:
sum(param.numel() for param in model.parameters())

1303111680

model size: 1.3B

model: 1.3G * 4 ~= 5.2G

gradient: 1.3G * 4 ~= 5.2G

optimizer: 1.3G * 4 * 2 ~= 10.4G

sum: 20.8G

## BitFit

In [12]:
# bitfit
# 选择模型参数里面的所有bias部分

num_param = 0
for name, param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        num_param += param.numel()

num_param

544768

In [13]:
num_param / sum(param.numel() for param in model.parameters())

0.000418051659240749

## Step5 配置训练参数

In [14]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)

## Step6 创建训练器

In [15]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

C:\Users\49207\AppData\Roaming\Python\Python311\site-packages\accelerate\accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Step7 模型训练

In [16]:
trainer.train()

  0%|          | 0/3357 [00:00<?, ?it/s]

{'loss': 2.9704, 'grad_norm': 1.1693657636642456, 'learning_rate': 4.985105749180816e-05, 'epoch': 0.0}
{'loss': 3.0886, 'grad_norm': 1.0056241750717163, 'learning_rate': 4.9702114983616324e-05, 'epoch': 0.01}
{'loss': 2.8747, 'grad_norm': 0.7882742285728455, 'learning_rate': 4.9553172475424484e-05, 'epoch': 0.01}
{'loss': 2.8072, 'grad_norm': 1.0763380527496338, 'learning_rate': 4.940422996723265e-05, 'epoch': 0.01}
{'loss': 2.7913, 'grad_norm': 1.0103319883346558, 'learning_rate': 4.925528745904081e-05, 'epoch': 0.01}
{'loss': 2.8424, 'grad_norm': 0.8298158049583435, 'learning_rate': 4.910634495084897e-05, 'epoch': 0.02}
{'loss': 2.6506, 'grad_norm': 0.8973371982574463, 'learning_rate': 4.895740244265713e-05, 'epoch': 0.02}
{'loss': 2.6084, 'grad_norm': 0.6329216957092285, 'learning_rate': 4.88084599344653e-05, 'epoch': 0.02}
{'loss': 2.5783, 'grad_norm': 1.1252562999725342, 'learning_rate': 4.865951742627346e-05, 'epoch': 0.03}
{'loss': 2.4904, 'grad_norm': 0.6707215905189514, 'lear

KeyboardInterrupt: 

In [20]:
model = model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
print(tokenizer.decode(model.generate(**ipt, max_length=128, do_sample=True)[0], skip_special_tokens=True))

Human: 考试有哪些技巧？

Assistant: 考试技巧：首先需要保持良好的心态，良好的心情，积极的备考态度，不要轻易去怀疑自己的实力，不要给自己太多的压力，相信自己一定能将所有的知识都考全及格，而且一定能考出满意的结果。


## Step8 模型推理

In [21]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [26]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
res = pipe(ipt, max_length=256, do_sample=True, num_return_sequences=5)

In [27]:
for item in res:
    print(item)
    print('--'*30)

{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 答案很多，这里只介绍一点：\n1、 阅读和回答问题的时间：从晚上开始写题最好，上午是回答问题的时间，可以减少焦虑，晚上可阅读文章，了解更多信息来帮助自己答题，考试中的每分钟都可以让大脑进行思考和理解知识。\n2、 写作：写好作文是一个很重要的技巧，一篇清晰的作文能让考官阅读和更全面的了解你的内容。'}
------------------------------------------------------------
{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 英语考试通常有以下注意事项。\n1、注意词汇量和语法：考试中的词汇量应达到一定程度，语法题目的准确性和简明性是考试重点。如果语法知识匮乏，可能会影响对答题的效率，还会出现答题错误，导致试题无章可循、内容难懂。因此，考生应熟练使用并准确无误地运用英语词汇、语法知识。\n2、保持稳定心态：紧张的考试心态会对考生带来消极影响，因此考生应保持积极乐观的心情，以良好的心态应对考试。'}
------------------------------------------------------------
{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧主要有两个要点，一是要了解、熟知答题的基本步骤和流程；二是要了解考试的考场规则。其中，考试的流程一般按照以下步骤：\n1. 作答之前\n1.1 准备、阅读题目的内容、格式以及标准答案：\n阅读题目的内容和格式要求\n理解题目的要求，比如，写明考查对象，考查要求是否严格，要求的内容与题目描述是否一致等。\n了解题目的标准答案：\n考试标准答案是对试题的正确答案进行规范描述和说明，便于答题和指导。'}
------------------------------------------------------------
{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 技巧1：保持良好的学习心态\n\nAssistant：要保持一个良好的学习心态，最重要的是控制自己的自制力。不要一遇到事情就急躁，先冷静下来